In [1]:
%load_ext autoreload
%autoreload 2

%cd ../..
!hostname

/p/fastdata/pli/Private/oberstrass1/datasets/vervet1818-3d
jrlogin07.jureca


In [2]:
import sys, os
import re

import h5py as h5
import numpy as np
import pandas as pd

import imageio
from tqdm import tqdm

import pli
import pli.image as im

In [67]:
wm_path = "data/aa/volume/wm/wm_4.h5"
volume_group = "volume"
affine_name = 'affine'

###

with h5.File(wm_path, 'r') as f:
    volume = f[volume_group][:]
    affine = f[affine_name][:]
    chunks = f[volume_group].chunks

print(volume.shape)
print("Affine\n", affine)

(1795, 243, 1942)
Affine
 [[0.0210944 0.        0.        0.       ]
 [0.        0.06      0.        0.       ]
 [0.        0.        0.0210944 0.       ]]


In [68]:
# Smooth in stack axis and remove small tissue components

median_kernel = (1, 5, 1)
tissue_cc_thresh = 0.001

###

from scipy import ndimage as ndi

# Smooth volume in z direction
median_volume = ndi.median_filter(volume, size=median_kernel)

tissue_labeled, _ = ndi.label(median_volume)
tissue_cc_counts = np.bincount(tissue_labeled.ravel())

tissue_cc_counts[0] = 0  # Background
largest_tissue_cc = np.max(tissue_cc_counts)
print(f"Largest tissue component: {largest_tissue_cc}")

large_cc_mask = (tissue_cc_counts >= largest_tissue_cc * tissue_cc_thresh)
print(f"Tissue components: {large_cc_mask.sum()}")
tissue_mask = large_cc_mask[tissue_labeled]

result_volume = tissue_mask > 0

Largest tissue component: 67724819
Tissue components: 1


In [69]:
# Save volume

out_file = "data/aa/volume/wm/wm_4_smooth.h5"
compression = 'gzip'

###

spacing = np.diag(affine)

with h5.File(out_file, 'w') as f:

    f[affine_name] = affine

    print(f"Save volume with chunk size {chunks} and affine matrix\n{affine}...")
    # Create empty dataset
    v_ds = f.create_dataset(
        name=volume_group,
        data=result_volume,
        chunks=chunks,
        compression=compression,
    )
    v_ds.attrs['spacing'] = spacing

print(f"Volume saved at {out_file}")

Save volume with chunk size (64, 64, 64) and affine matrix
[[0.0210944 0.        0.        0.       ]
 [0.        0.06      0.        0.       ]
 [0.        0.        0.0210944 0.       ]]...
Volume saved at data/aa/volume/wm/wm_4_smooth.h5
